In [2]:
from scipy.io import loadmat
import numpy as np

In [3]:
X_raw = loadmat("R2198_20ms.mat")
y_raw = np.loadtxt("R2198_locations.dat")

X_raw = X_raw['mm'].T

In [75]:
def generate_datasets(window=50, step=25, test_set_sampling='chunk', test_set_size=0.2, test_chunk_location=0.8):
    print('generating data')
    print('window {}, step {}'.format(window,step))
    dataset_length = int(X_raw.shape[0]/step)
    X = np.zeros((dataset_length, 2*window*33))
    y = np.zeros((dataset_length, 2))

    for filtered_i, data_i in enumerate(range(window, X_raw.shape[0]-window, step)):
        X[filtered_i] = np.real(np.fft.fft(X_raw[data_i-window:data_i+window].T)).flatten()
        y[filtered_i] = y_raw[data_i]

#     X = X[window:-window]
#     y = y[window:-window]
    if test_set_sampling == 'uniform_chunks':
        print('uniform chunks')
        # refer to report for how this is calculated.         
        test_set_size = 0.1
        chunk_size = int(2*window/step)
        test_set_length = int(dataset_length*test_set_size)
        chunks_count = int(test_set_length/chunk_size)
        chunk_step = int((dataset_length-chunk_size)/chunks_count)
        test_chunks = [list(range(i*chunk_step, i*chunk_step + chunk_size)) for i in range(chunks_count)]
        discard_chunks = [list(range(i*chunk_step - chunk_size, i*chunk_step)) for i in range(chunks_count)][1:]
        # flatten chunk lists
        test_chunks = sum(test_chunks, [])
        discard_chunks = sum(discard_chunks, [])
        test_and_discard_chunks = test_chunks + discard_chunks
        X_test = X[test_chunks]
        X_train = np.delete(X, test_and_discard_chunks, axis=0)
        y_test = y[test_chunks]
        y_train = np.delete(y, test_and_discard_chunks, axis=0)
#         print(test_chunks)
#         print(discard_chunks)
#         print(test_and_discard_chunks)
#         print(sum(chunks, []))
#         return
    if test_set_sampling == 'chunk':
        print('sampling with a chunk, size {}, location {}'.format(test_set_size, test_chunk_location))
        test_set_start = int(dataset_length*test_chunk_location)
        test_set_end = test_set_start + int(dataset_length*test_set_size)
        print('indexes {}:{}'.format(test_set_start, test_set_end))
        test_chunk = np.arange(test_set_start, test_set_end)
        X_test = X[test_chunk]
        X_train = np.delete(X, test_chunk, axis=0)
        y_test = y[test_chunk]
        y_train = np.delete(y, test_chunk, axis=0)
    if test_set_sampling == 'random':
        print('random test set, probably not a good option here')
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_set_size, random_state=0)
    print(X_test.shape, X_train.shape, y_test.shape, y_train.shape)
    return X_test, X_train, y_test, y_train

X_test, X_train, y_test, y_train = generate_datasets(test_set_sampling='uniform_chunks')

generating data
window 50, step 25
uniform chunks
(256, 3300) (2058, 3300) (256, 2) (2058, 2)


In [76]:
X_test, X_train, y_test, y_train = generate_datasets(test_set_sampling='uniform_chunks')

generating data
window 50, step 25
uniform chunks
(256, 3300) (2058, 3300) (256, 2) (2058, 2)


In [63]:
asd = np.arange(10)

ert = np.delete(asd, [9, 1])
asd, ert[[1,2,3]]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([2, 3, 4]))

In [28]:
np.arange(4,10)

array([4, 5, 6, 7, 8, 9])

In [67]:
np.arange(9).reshape((3,3))[np.array([0,2])]

array([[0, 1, 2],
       [6, 7, 8]])